# DiffDock
Dock a PDB files and a SMILES with [DiffDock](https://github.com/gcorso/DiffDock).

Select Runtime / Run all to run an example PDB file and SMILES.

v2 improvements:
- works on proteins >1000 aas (with flag added to extract.py)
- works on standard GPU (by reducing batch size from 10 to 6)
- works with AlphaFold ids (AF-...) as well as PDB ids
- works with comma-delimited PDB_ids and/or SMILES
- runs smina to generate affinities (as DiffDock posed, or with smina minimization)
- shows results in a py3DMol view

v3:
- fix py3dmol incompatibility
- downgrade pytorch since the colab version takes forever to install pytorch_geometric (no binary so it has to compile)

v4:
- replace smina with the more accurate gnina
- remove the annoying run twice thing now there is a binary

v5:
- add ProLIF fingerprint

v6:
- bugfixes due to ProLIF version changes. prolif is now locked at 2.0.1

colab by [@btnaughton](https://twitter.com/btnaughton)

In [1]:
#@title PDB + SMILES input

PDB_id = '' #@param {type:"string"}
SMILES_or_pubchem_id = 'C[C@H]1/C=C/C=C(\\C(=O)NC2=CC(=C3C(=C2O)C(=C(C4=C3C(=O)[C@](O4)(O/C=C/[C@@H]([C@H]([C@H]([C@@H]([C@@H]([C@@H]([C@H]1O)C)O)C)OC(=O)C)C)OC)C)C)O)O)/C' #@param {type:"string"}

#@markdown Download a tar file containing all results?
download_results = True #@param {type:"boolean"}

In [2]:
import os
import ipywidgets as widgets
from IPython.display import display

import os
import ipywidgets as widgets
from IPython.display import display

# Global variable to store the path of the uploaded file
uploaded_pdb_path = None

def upload_pdb_file():
    """Upload PDB file using a widget and save it to a specified directory."""
    global uploaded_pdb_path

    PDB_DIR = "/tmp/pdb/"
    os.makedirs(PDB_DIR, exist_ok=True)

    uploader = widgets.FileUpload(
        accept='.pdb',
        multiple=False
    )
    display(uploader)

    def on_upload_change(change):
        global uploaded_pdb_path
        if uploader.value:
            uploaded_filename = next(iter(uploader.value))
            content = uploader.value[uploaded_filename]['content']
            file_path = os.path.join(PDB_DIR, uploaded_filename)
            with open(file_path, 'wb') as f:
                f.write(content)
            uploaded_pdb_path = file_path
            print(f"File uploaded successfully: {file_path}")

    uploader.observe(on_upload_change, names='value')

    return uploader

# Call the upload function
uploader = upload_pdb_file()

if uploaded_pdb_path:
    pdb_files = [uploaded_pdb_path]
    # Process the uploaded PDB file as needed
else:
    print("No PDB file uploaded.")

FileUpload(value={}, accept='.pdb', description='Upload')

No PDB file uploaded.


In [3]:
import os
import requests
import time
from random import random

# def download_pdb_file(pdb_id: str) -> str:
#     """Download pdb file as a string from rcsb.org"""
#     PDB_DIR ="/tmp/pdb/"
#     os.makedirs(PDB_DIR, exist_ok=True)

#     # url or pdb_id
#     if pdb_id.startswith('http'):
#         url = pdb_id
#         filename = url.split('/')[-1]
#     elif pdb_id.endswith(".pdb"):
#         return pdb_id
#     else:
#         if pdb_id.startswith("AF"):
#             url = f"https://alphafold.ebi.ac.uk/files/{pdb_id}-model_v3.pdb"
#         else:
#             url = f"http://files.rcsb.org/view/{pdb_id}.pdb"
#         filename = f'{pdb_id}.pdb'

#     cache_path = os.path.join(PDB_DIR, filename)
#     if os.path.exists(cache_path):
#         return cache_path

#     pdb_req = requests.get(url)
#     pdb_req.raise_for_status()
#     open(cache_path, 'w').write(pdb_req.text)
#     return cache_path

def download_smiles_str(pubchem_id: str, retries:int = 2) -> str:
    """Given a pubchem id, get a smiles string"""
    while True:
        req = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/{pubchem_id}/property/CanonicalSMILES/CSV")
        smiles_url_csv = req.text if req.status_code == 200 else None
        if smiles_url_csv is not None:
            break
        if retries == 0:
            return None
        time.sleep(1+random())
        retries -= 1

    return smiles_url_csv.splitlines()[1].split(',')[1].strip('"').strip("'") if smiles_url_csv is not None else None

In [4]:
# if not PDB_id or not SMILES_or_pubchem_id:
#     PDB_id = "2HV5"
#     SMILES_or_pubchem_id = "C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O" # quercetin
#     print(f"No input supplied. Using example data: {PDB_id} and {SMILES_or_pubchem_id}")

# to run many PDB+smiles at once, fill in a list of PDB_files and smiles here...
# pdb_files = [download_pdb_file(_PDB_id) for _PDB_id in PDB_id.split(",")]
pdb_files = ["/tmp/pdb/STAT3_AlphaFold.pdb"]
smiless = [download_smiles_str(_SMILES_or_pubchem_id) if str(_SMILES_or_pubchem_id).isnumeric() else _SMILES_or_pubchem_id
           for _SMILES_or_pubchem_id in SMILES_or_pubchem_id.split(',') ]

with open("/tmp/input_protein_ligand.csv", 'w') as out:
    out.write("protein_path,ligand\n")
    for pdb_file in pdb_files:
        for smiles in smiless:
            out.write(f"{pdb_file},{smiles}\n")

In [5]:
pdb_files

['/tmp/pdb/STAT3_AlphaFold.pdb']

In [ ]:
# import os
# import requests
# import time
# from random import random

# def download_pdb_file(pdb_id: str) -> str:
#     """Download pdb file as a string from rcsb.org"""
#     PDB_DIR ="/tmp/pdb/"
#     os.makedirs(PDB_DIR, exist_ok=True)

#     # url or pdb_id
#     if pdb_id.startswith('http'):
#         url = pdb_id
#         filename = url.split('/')[-1]
#     elif pdb_id.endswith(".pdb"):
#         return pdb_id
#     else:
#         if pdb_id.startswith("AF"):
#             url = f"https://alphafold.ebi.ac.uk/files/{pdb_id}-model_v3.pdb"
#         else:
#             url = f"http://files.rcsb.org/view/{pdb_id}.pdb"
#         filename = f'{pdb_id}.pdb'

#     cache_path = os.path.join(PDB_DIR, filename)
#     if os.path.exists(cache_path):
#         return cache_path

#     pdb_req = requests.get(url)
#     pdb_req.raise_for_status()
#     open(cache_path, 'w').write(pdb_req.text)
#     return cache_path

# def download_smiles_str(pubchem_id: str, retries:int = 2) -> str:
#     """Given a pubchem id, get a smiles string"""
#     while True:
#         req = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/{pubchem_id}/property/CanonicalSMILES/CSV")
#         smiles_url_csv = req.text if req.status_code == 200 else None
#         if smiles_url_csv is not None:
#             break
#         if retries == 0:
#             return None
#         time.sleep(1+random())
#         retries -= 1

#     return smiles_url_csv.splitlines()[1].split(',')[1].strip('"').strip("'") if smiles_url_csv is not None else None

In [ ]:
# if not PDB_id or not SMILES_or_pubchem_id:
#     PDB_id = "6agt"
#     SMILES_or_pubchem_id = "COc(cc1)ccc1C#N"
#     print(f"No input supplied. Using example data: {PDB_id} and {SMILES_or_pubchem_id}")

# # to run many PDB+smiles at once, fill in a list of PDB_files and smiles here...
# pdb_files = [download_pdb_file(_PDB_id) for _PDB_id in PDB_id.split(",")]
# smiless = [download_smiles_str(_SMILES_or_pubchem_id) if str(_SMILES_or_pubchem_id).isnumeric() else _SMILES_or_pubchem_id
#            for _SMILES_or_pubchem_id in SMILES_or_pubchem_id.split(',') ]

# with open("/tmp/input_protein_ligand.csv", 'w') as out:
#     out.write("protein_path,ligand\n")
#     for pdb_file in pdb_files:
#         for smiles in smiless:
#             out.write(f"{pdb_file},{smiles}\n")

In [6]:
# clear out old results if running multiple times -- hopefully they have been downloaded already
!rm -rf /content/DiffDock/results

## Install prerequisites

In [7]:
!pip install ipython-autotime --quiet
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.0 MB/s eta 0:00:00
time: 293 µs (started: 2024-10-22 02:05:00 +00:00)


In [8]:
if not os.path.exists("/content/DiffDock"):
    %cd /content
    !git clone https://github.com/gcorso/DiffDock.git
    %cd /content/DiffDock
    !git checkout a6c5275 # remove/update for more up to date code

/content
Cloning into 'DiffDock'...
remote: Enumerating objects: 517, done.
remote: Counting objects: 100% (511/511), done.
remote: Compressing objects: 100% (318/318), done.
remote: Total 517 (delta 244), reused 400 (delta 191), pack-reused 6 (from 1)
Receiving objects: 100% (517/517), 233.08 MiB | 47.31 MiB/s, done.
Resolving deltas: 100% (244/244), done.
/content/DiffDock
Note: switching to 'a6c5275'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at a6c5275 remove debugging raise in the inference file
time: 1

In [9]:
try:
    import biopandas
except:
    !pip install pyg==0.7.1 --quiet
    !pip install pyyaml==6.0 --quiet
    !pip install scipy==1.7.3 --quiet
    !pip install networkx==2.6.3 --quiet
    !pip install biopython==1.79 --quiet
    !pip install rdkit-pypi==2022.03.5 --quiet
    !pip install e3nn==0.5.0 --quiet
    !pip install spyrmsd==0.5.2 --quiet
    !pip install pandas==1.5.3 --quiet
    !pip install biopandas==0.4.1 --quiet
    # not diffdock-specific:
    !pip install prolif==2.0.1 --quiet # no real version??
    !pip install py3dmol==2.0.3 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 52.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.22.4 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.22.4 which is incompatible.
albumentations 1.4.15 requires scipy>=1.10.0, but you have scipy 1.7.3 which is incompatible.
arviz 0.19.0 requires numpy>=1.23.0, but you have numpy 1.22.4 which is

In [ ]:
import torch
print(torch.__version__)

try:
    import torch_geometric
except ModuleNotFoundError:
    !pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
    !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
    !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
    !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
    !pip install git+https://github.com/pyg-team/pytorch_geometric.git  --quiet # @ 15573f4674b2a37b1b9adc967df69ef6eee573ea

2.4.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Install ESM and prepare PDB file for ESM

In [ ]:
if not os.path.exists("/content/DiffDock/esm"):
    %cd /content/DiffDock
    !git clone https://github.com/facebookresearch/esm
    %cd /content/DiffDock/esm
    !git checkout ca8a710 # remove/update for more up to date code
    !sudo pip install -e .
    %cd /content/DiffDock

In [ ]:
%cd /content/DiffDock
!python datasets/esm_embedding_preparation.py --protein_ligand_csv /tmp/input_protein_ligand.csv --out_file data/prepared_for_esm.fasta

In [ ]:
%cd /content/DiffDock
%env HOME=esm/model_weights
%env PYTHONPATH=$PYTHONPATH:/content/DiffDock/esm
!python /content/DiffDock/esm/scripts/extract.py esm2_t33_650M_UR50D data/prepared_for_esm.fasta data/esm2_output --repr_layers 33 --include per_tok --truncation_seq_length 30000

## Run DiffDock

In [ ]:
%cd /content/DiffDock
!python -m inference --protein_ligand_csv /tmp/input_protein_ligand.csv --out_dir results/user_predictions_small --inference_steps 20 --samples_per_complex 40 --batch_size 6

# Post-process and download results

In [ ]:
%cd /content/DiffDock
!wget https://sourceforge.net/projects/smina/files/smina.static/download -O smina && chmod +x smina
!wget https://github.com/gnina/gnina/releases/download/v1.0.3/gnina -O gnina && chmod +x gnina

## Run gnina to estimate affinity

In [ ]:
import re
import pandas as pd
from glob import glob
from shlex import quote
from datetime import datetime
from tqdm.auto import tqdm
from google.colab import files

%cd /content/DiffDock/results/user_predictions_small
results_dirs = glob("./index*")

rows = []
for results_dir in tqdm(results_dirs, desc="runs"):
    results_pdb_file = "/tmp/pdb/" + re.findall("tmp-pdb-(.+\.pdb)", results_dir)[0]
    results_smiles = re.findall("pdb_+(.+)", results_dir)[0]
    results_sdfs = [os.path.join(results_dir, f) for f in os.listdir(results_dir) if "confidence" in f and f.endswith(".sdf")]

    results_pdb_file_no_hetatms = f"{results_pdb_file}_nohet.pdb"
    !grep -v "^HETATM" {results_pdb_file} > {results_pdb_file_no_hetatms}
    !cp {results_pdb_file} .

    for results_sdf in tqdm(results_sdfs, leave=False, desc="files"):
        confidence = re.findall("confidence([\-\.\d]+)\.sdf", results_sdf)[0]

        scored_stdout = !/content/DiffDock/gnina --score_only -r "{results_pdb_file_no_hetatms}" -l "{results_sdf}"
        scored_affinity = re.findall("Affinity:\s*([\-\.\d+]+)", '\n'.join(scored_stdout))[0]
        minimized_stdout = !/content/DiffDock/gnina --local_only --minimize -r "{results_pdb_file_no_hetatms}" -l "{results_sdf}" --autobox_ligand "{results_sdf}" --autobox_add 2
        minimized_affinity = re.findall("Affinity:\s*([\-\.\d+]+)", '\n'.join(minimized_stdout))[0]

        rows.append((results_pdb_file.split('/')[-1], results_smiles, float(confidence), float(scored_affinity), float(minimized_affinity), results_sdf))

df_results = pd.DataFrame(rows, columns=["pdb_file", "smiles", "diffdock_confidence", "gnina_scored_affinity", "gnina_minimized_affinity", "sdf_file"])
df_results_tsv = "df_diffdock_results.tsv"
df_results.to_csv(df_results_tsv, sep='\t', index=None)

### Top hit by DiffDock confidence for any PDB/smiles combination

In [ ]:
top_hit = df_results.sort_values("diffdock_confidence", ascending=False).iloc[0]
df_results.sort_values("diffdock_confidence", ascending=False).head(3)

## Show prolif fingerprint

In [ ]:
import prolif as plf
from prolif.plotting.network import LigNetwork
from rdkit import Chem

# load protein
prot = Chem.MolFromPDBFile(top_hit.pdb_file, removeHs=False)
prot = plf.Molecule(prot)

# load ligands
lig_suppl = list(plf.sdf_supplier(top_hit.sdf_file))

# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)

df_prolif = fp.to_dataframe()
display(df_prolif)

prolif_fingerprint_html = "prolif_fingerprint.html"
net = LigNetwork.from_fingerprint(fp, lig_suppl[0], kind="frame", frame=0, rotation=0)
net.save(prolif_fingerprint_html)
net.display()

## Create dataframe, tar file and download

In [ ]:
out_pdbs = ' '.join(set(df_results.pdb_file.apply(quote)))
out_sdfs = ' '.join(df_results.sdf_file.apply(quote))

if download_results:
    tarname = f"diffdock_{datetime.now().isoformat()[2:10].replace('-','')}"
    _ = !tar cvf {tarname}.tar --transform 's,^,{tarname}/,' --transform 's,\./,,' {out_pdbs} {out_sdfs} {df_results_tsv} {prolif_fingerprint_html}

    files.download(f"{tarname}.tar")

## Compare gnina affinities with DiffDock confidences

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import linregress
%config InlineBackend.figure_format='retina'

for (pdb_file, smiles), df_group in df_results.groupby(["pdb_file", "smiles"]):
    f, ax = plt.subplots(1, 2, figsize=(20,8))
    sns.regplot(data=df_group, x="diffdock_confidence", y="gnina_scored_affinity", ax=ax[0]);
    sns.regplot(data=df_group, x="diffdock_confidence", y="gnina_minimized_affinity", ax=ax[1]);

    slope, intercept, r_value_scored, p_value, std_err = linregress(df_group["diffdock_confidence"], df_group["gnina_scored_affinity"])
    slope, intercept, r_value_minimized, p_value, std_err = linregress(df_group["diffdock_confidence"], df_group["gnina_minimized_affinity"])
    ax[0].set_title(f"{pdb_file} {smiles[:30]} gnina scored r={r_value_scored:.3f}");
    ax[1].set_title(f"{pdb_file} {smiles[:30]} gnina minimized r={r_value_minimized:.3f}");

# Visualize top hit (highest confidence) in 3D

In [ ]:
from IPython.display import HTML
import py3Dmol

resid_hover = """
function(atom,viewer) {
    if(!atom.label) {
        atom.label = viewer.addLabel(atom.chain+" "+atom.resn+" "+atom.resi,
            {position: atom, backgroundColor: 'mintcream', fontColor:'black', fontSize:12});
    }
}"""
unhover_func = """
function(atom,viewer) {
    if(atom.label) {
        viewer.removeLabel(atom.label);
        delete atom.label;
    }
}"""

view = py3Dmol.view(width=800, height=800)
view.setCameraParameters({'fov': 35, 'z': 100});


# add sdf
view.addModel(open(top_hit.sdf_file).read(), "sdf")
view.setStyle({"model": 0}, {'stick':{"color":"#ff0000"}})
view.setViewStyle({"model": 0}, {'style':'outline','color':'black','width':0.1})
view.zoomTo();

# add pdb
view.addModel(open(top_hit.pdb_file).read(), "pdb");
view.setStyle({"model": 1}, {"cartoon":{"color":"spectrum"}})
view.setStyle({"model": 1, "hetflag":True}, {'stick':{"color":"spectrum"}})

model = view.getModel()
model.setHoverable({}, True, resid_hover, unhover_func)

view